## B-cubed assets example

In [1]:
import requests
import json

from dataspace_apis import *

### Vars setup

In [4]:

#PROVIDER_URL = "https://consumer-edc-connector.apps.dcw1.paas.psnc.pl"
#CONSUMER_URL = "https://ad4gd-provider-edc.dashboard-siba.store"
#FEDERATED_CATALOG_BASE_URL = "https://ad4gd-catalog-edc.dashboard-siba.store"
#CONSUMER_BACKEND_URL = "https://consback-edc-connector.apps.dcw1.paas.psnc.pl"

PROVIDER_CONNECTOR_ID = "provider"

PROVIDER_URL = "https://provider-edc-connector.apps.paas-dev.psnc.pl"
CONSUMER_URL = "https://consumer-edc-connector.apps.paas-dev.psnc.pl"
FEDERATED_CATALOG_BASE_URL = "https://federatedcatalog-edc-connector.apps.paas-dev.psnc.pl"
CONSUMER_BACKEND_URL = "https://consback-edc-connector.apps.paas-dev.psnc.pl"

PROVIDER_API = f"{PROVIDER_URL}/api"
PROVIDER_CONTROL = f"{PROVIDER_URL}/control"
PROVIDER_MANAGEMENT = f"{PROVIDER_URL}/management"
PROVIDER_PROTOCOL = f"{PROVIDER_URL}/protocol"
PROVIDER_PUBLIC = f"{PROVIDER_URL}/public"

CONSUMER_API = f"{CONSUMER_URL}/api"
CONSUMER_CONTROL = f"{CONSUMER_URL}/control"
CONSUMER_MANAGEMENT = f"{CONSUMER_URL}/management"
CONSUMER_PROTOCOL = f"{CONSUMER_URL}/protocol"
CONSUMER_PUBLIC = f"{CONSUMER_URL}/public"

CONSUMER_BACKEND_EDR = f"{CONSUMER_BACKEND_URL}/edr-endpoint"
FEDERATED_CATALOG_URL = f"{FEDERATED_CATALOG_BASE_URL}/catalog"

provider_control_internal = PROVIDER_CONTROL
provider_public_internal = PROVIDER_PUBLIC
provider_protocol_internal = PROVIDER_PROTOCOL

default_headers = {
    "Content-Type": "application/json",
    "x-api-key": "edc",
}

### Conn Check

In [7]:
def check_health(coreApiUrl):
    print(f"{coreApiUrl}/check/health/")
    rp = requests.get(f"{coreApiUrl}/check/health/", headers=default_headers).json()
    print(rp)

check_health(PROVIDER_API)
check_health(CONSUMER_API)
print("They're Alive!")

https://provider-edc-connector.apps.paas-dev.psnc.pl/api/check/health/
{'componentResults': [{'failure': None, 'component': 'Dataplane Self Registration', 'isHealthy': True}, {'failure': None, 'component': 'BaseRuntime', 'isHealthy': True}], 'isSystemHealthy': True}
https://consumer-edc-connector.apps.paas-dev.psnc.pl/api/check/health/
{'componentResults': [{'failure': None, 'component': 'Dataplane Self Registration', 'isHealthy': True}, {'failure': None, 'component': 'BaseRuntime', 'isHealthy': True}], 'isSystemHealthy': True}
They're Alive!


### Populating the provider

In [29]:
asset_id = "B-Cubed_bcdbb4e2-b9eb-448d-9467-733e744f2b9a-without-GBIF"
asset_name = "Digital Catalogue of Biodiversity of Poland — Insecta: Strepsiptera (without GBIF)"
asset_url = "https://api.gbif.org/v1/occurrence/download/request"
# will be passed during a transfer as proxy body
asset_query_json = '''
{
  "sendNotification": false,
  "format": "SQL_TSV_ZIP",
  "sql": "
  SELECT
    \"year\",
    classKey,
    class,
    speciesKey,
    species,
    COUNT(*) AS occurrences,
    MIN(COALESCE(coordinateUncertaintyInMeters, 1000)) AS minCoordinateUncertaintyInMeters,
    IF(ISNULL(classKey), NULL, SUM(COUNT(*)) OVER (PARTITION BY classKey)) AS classCount
  FROM
    occurrence
  WHERE
    occurrenceStatus = 'PRESENT'
    AND speciesKey IN (4480653, 4480642, 4480637, 4480628, 4480626, 4480624, 9719065)
    AND continent = 'EUROPE'
    AND \"year\" >= 1900
    AND hasCoordinate = TRUE
    AND speciesKey IS NOT NULL
    AND NOT ARRAY_CONTAINS(issue, 'ZERO_COORDINATE')
    AND NOT ARRAY_CONTAINS(issue, 'COORDINATE_OUT_OF_RANGE')
    AND NOT ARRAY_CONTAINS(issue, 'COORDINATE_INVALID')
    AND NOT ARRAY_CONTAINS(issue, 'COUNTRY_COORDINATE_MISMATCH')
    AND (
      LOWER(identificationVerificationStatus) NOT IN (
        'unverified',
        'unvalidated',
        'not validated',
        'under validation',
        'not able to validate',
        'control could not be conclusive due to insufficient knowledge',
        'uncertain',
        'unconfirmed',
        'unconfirmed - not reviewed',
        'validation requested'
        )
      OR identificationVerificationStatus IS NULL
    )
  GROUP BY
    \"year\",
    classKey,
    class,
    speciesKey,
    species
  ORDER BY
    \"year\" DESC,
    speciesKey ASC;
  "
}
'''

In [31]:
import base64

# b-cubed username:password
credentials = "dparkhanovich:FeliksHub444/"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

def create_bcubed_asset(
    asset_id: str,
    asset_name: str,
    management_url: str,
    baseUrl: str
):
    return requests.post(
        data=json.dumps(
            {
                "@context": {"edc": "https://w3id.org/edc/v0.0.1/ns/"},
                "@id": asset_id,
                "properties": {"name": asset_id, "contenttype": "application/json"},
                "private_properties": {"name": asset_id, "contenttype": "application/json"},
                "dataAddress": {
                    "name": asset_name, 
                    "baseUrl": baseUrl, 
                    "type": "HttpData",
                    "proxyMethod": "true",
                    "proxyBody": "true",
                    "header:authorization": f"Basic {encoded_credentials}",
                },
            }
        ),
        url=f"{management_url}/v3/assets",
        headers=default_headers
    )

asset = create_bcubed_asset(asset_id, asset_name, PROVIDER_MANAGEMENT, asset_url)

# expected 200 or 409 (already exists)
print(asset.status_code)
print(asset.text)

409
[{"message":"Asset with ID B-Cubed_bcdbb4e2-b9eb-448d-9467-733e744f2b9a-without-GBIF already exists","type":"ObjectConflict","path":null,"invalidValue":null}]


In [18]:
policy_id = "test-policy-001"

policy = create_policy(policy_id, PROVIDER_MANAGEMENT, default_headers, permissions=[])

# expected 200 or 409 (already exists) because the @id is fixed in the example
print(policy.status_code)

409


Contract definition - Links an asset with a policy

In [20]:
contract_definition_id = "test-contract-definition-001"

contract_definition = create_contract_definition(contract_definition_id, PROVIDER_MANAGEMENT, asset_id, policy_id, default_headers)

# expected 200 or 409 (already exists) because the @id is fixed in the example
contract_definition.status_code

409

### Working on consumer side

Fetch catalog, negotiate and perform transfer

**Local Transfer**

Stage 1 (get request id)

In [32]:
import time

# 1. fetch catalog
fetched_catalog = fetch_catalog(FEDERATED_CATALOG_URL, default_headers)

print(f"1. Catalog fetch: {fetched_catalog.status_code}")

# 2. check if there is an existing negotiation
existing_negotiation = check_existing_negotiation(asset_id, CONSUMER_MANAGEMENT, default_headers)

print(f"Negotiation for asset \"{asset_id}\": {existing_negotiation}")
contract_agreement_id = ""

if existing_negotiation is not None:
    # 2A. Use existing negotiation
    contract_agreement_id = existing_negotiation
else:
    offer_id = get_offer_id(fetched_catalog.json(), asset_id)

    print(offer_id)

    # 2. negotiate a contract
    negotiated_contract = negotiate_contract(
        offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [], default_headers)
    
    # If there's policy with rules then negotiated_contract = negotiate_contract(offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [allowed_region_rule], default_headers)

    print(f"2. Negotiate a contract: {negotiated_contract.status_code}")
    print(negotiated_contract.json())

    # wait a minute until the negotiation will be finalized (automatic interval process takes maximum 60 secondes to load new contracts)
    print("... Waiting for negotiation to save ...")
    time.sleep(60)

    contract_negotiation_id = negotiated_contract.json()["@id"]
    contract_agreement = get_contract_agreement_id(contract_negotiation_id, CONSUMER_MANAGEMENT, default_headers)
    print(contract_agreement.json())

    if ("contractAgreementId" in contract_agreement.json()):
        contract_agreement_id = contract_agreement.json()["contractAgreementId"]

print("[@ContractAgreementId]:", contract_agreement_id)

# 3. transfer asset (this will save the asset to PSNC MinIO)
query_params = "?url=false"
consumer_backend_url_with_query_params = CONSUMER_BACKEND_EDR + query_params

# Check for existing transfer for given asset and agreement
existing_transfer = check_existing_transfer(asset_id, contract_agreement_id, consumer_backend_url_with_query_params, CONSUMER_MANAGEMENT, default_headers)

# 3. transfer asset
if existing_transfer is not None:
    print(f"Requested transfer already exists: {existing_transfer}")
    
    requested_transfer = get_transfer(existing_transfer, CONSUMER_MANAGEMENT, default_headers)
else:
    requested_transfer = request_consumer_pull_transfer(
        PROVIDER_CONNECTOR_ID,
        CONSUMER_MANAGEMENT,
        consumer_backend_url_with_query_params,
        provider_protocol_internal,
        contract_agreement_id,
        default_headers
    )

print("... Waiting for transfer to save ...")
time.sleep(10)

print(f"3. Transfer an asset: {requested_transfer.status_code}")
print(requested_transfer.json())

print("4. Get credentials")
print(f"{CONSUMER_MANAGEMENT}/v3/edrs/{requested_transfer.json()["@id"]}/dataaddress")
transfer_credentials = get_transfer_data_credentials(CONSUMER_MANAGEMENT, requested_transfer.json()["@id"], default_headers)
print(transfer_credentials)

public = f"{transfer_credentials['endpoint']}/"

headers = {"Authorization": transfer_credentials['authorization']}

data = requests.post(
    public,
    data=asset_query_json,
    headers=headers,
)

if (data.text):
    print(data.text)
else:
    print(data.content)
    

1. Catalog fetch: 200
Negotiation for asset "B-Cubed_bcdbb4e2-b9eb-448d-9467-733e744f2b9a-without-GBIF": 74b82a7e-caa8-4ca2-be96-918c13a8a97f
[@ContractAgreementId]: 74b82a7e-caa8-4ca2-be96-918c13a8a97f
Requested transfer already exists: 1666cc45-fa23-40e9-b11b-2c5a7b521cab
... Waiting for transfer to save ...
3. Transfer an asset: 200
{'@id': '1666cc45-fa23-40e9-b11b-2c5a7b521cab', '@type': 'TransferProcess', 'state': 'STARTED', 'stateTimestamp': 1751482925650, 'type': 'CONSUMER', 'callbackAddresses': {'@type': 'CallbackAddress', 'transactional': False, 'uri': 'https://consback-edc-connector.apps.paas-dev.psnc.pl/edr-endpoint?url=false', 'events': 'transfer.process.started'}, 'correlationId': 'a1a8f2c3-448d-4680-898d-b92e257786ec', 'assetId': 'B-Cubed_bcdbb4e2-b9eb-448d-9467-733e744f2b9a-without-GBIF', 'contractId': '74b82a7e-caa8-4ca2-be96-918c13a8a97f', 'transferType': 'HttpData-PULL', 'dataDestination': {'@type': 'DataAddress', 'type': 'HttpProxy'}, '@context': {'@vocab': 'https://

Stage 2 (download data using request id)